# Tensorflow自定义的模型和训练算法

## 12.3.1 自定义损失函数

In [1]:
import keras.layers
import tensorflow as tf
from sympy.assumptions.lra_satask import lra_satask
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1.0
    squared_loss = 0.5 * tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)


C:\Users\Administrator\anaconda3\envs\DeepLearning-py310\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


### 导入并处理数据

In [2]:
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42
)

### 创建并编译model

In [4]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Normalization(),
    keras.layers.Dense(256, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.LayerNormalization(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Normalization(),
    keras.layers.Dense(1, activation= "sigmoid")
])

model.compile(loss=huber_fn, optimizer="nadam")

early_stopping = EarlyStopping(patience=10)


model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

Epoch 1/20
363/363 [==============================] - 4s 3ms/step - loss: 0.7473 - val_loss: 0.7080
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080
Epoch 6/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080
Epoch 7/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080
Epoch 8/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080
Epoch 9/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080
Epoch 10/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7433 - val_loss: 0.7080

In [6]:
model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.7290


0.7289659976959229

### 保存model

In [8]:
model.save("my_custom_model.h5")

## 12.3.2 保存和加载包含自定义组件的模型

In [9]:
model = keras.models.load_model("my_custom_model.h5", custom_objects={"huber_fn": huber_fn})

### 实现不同阈值的损失函数

In [11]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = 0.5 * tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - 0.5 * threshold ** 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

### 使用新的损失函数编译model

In [12]:
model.compile(loss= create_huber, optimizer="nadam")

In [13]:
model.save("my_custom_model_threshold.h5")

In [ ]:
model = keras.models.load_model("my_custom_model_threshold.h5", custom_objects={"huber_fn": create_huber(0.5)})n

### 创建一个keras.losses.loss类的子类来实现加载模型的时候自动保存阈值

In [14]:
from tensorflow.keras.losses import Loss
class HuberLoss(Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2.0
        linear_loss = self.threshold * tf.abs(error) - 0.5 * tf.square(self.threshold)
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold} # **表示将字典中的键值对从源字典里面取出来作为参数传入函数

### 使用自定义的loss类来编译model

In [15]:
model.compile(loss=HuberLoss(threshold=2.), optimizer= "nadam")
model.save("My_custom_model_threshold2.h5")

In [16]:
model = keras.models.load_model("My_custom_model_threshold2.h5",custom_objects={"HuberLoss": HuberLoss})

## 12.3.3 自定义激活函数 初始化 正则化器

In [17]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2.0 / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(weights)) * 1e-3

def my_positive_weights(weights):
    return tf.where(weights < 0, tf.zeros_like(weights), weights)

layer = keras.layers.Dense(10, activation=my_softplus, kernel_initializer=my_glorot_initializer, kernel_regularizer=my_l1_regularizer, bias_regularizer=my_l1_regularizer, activity_regularizer=my_l1_regularizer, kernel_constraint=my_positive_weights, bias_constraint=my_positive_weights)

### 如果函数有需要和模型一起保存的参数 则需要继承相应的类
#### 必须为损失 层 模型实现call方法 或者为正则化初始化和约束实现__call__方法

In [ ]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(weights)) * self.factor

    def get_config(self):
        return {"factor": self.factor}

## 12.3.4自定义指标

### 定义一个流式度量 即每次返回的是基于此刻的预测总指标

In [18]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):

    # 初始化函数，设置阈值和Huber函数
    # threshold: Huber函数的阈值，默认值为1.0
    # **kwargs: 其他传递给父类的参数
        super().__init__(**kwargs)  # 调用父类的初始化方法
    # 设置Huber函数的阈值
        self.threshold = threshold
    # 创建Huber函数实例
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight(name="total", initializer="zeros") # 每次计算指标时都会更新这个变量
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):

        # 计算损失，使用huber_fn函数计算真实值和预测值之间的损失
        loss = self.huber_fn(y_true, y_pred)
        # 更新指标
        self.total.assign_add(tf.reduce_sum(loss))
        self.count.assign_add(tf.cast(tf.size(loss), tf.float32))

    def result(self):
    # 计算并返回平均值
    # 通过将总数除以数量得到平均值
        return self.total / self.count

    def get_config(self):

        return {"threshold": self.threshold}  # 返回包含阈值的配置字典

## 12.3.5自定义层

### 创建keras.layers.Layer的子类来实现自定义层

In [19]:
class MyLayer(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name="kernel", shape=[batch_input_shape[-1], self.units], initializer="glorot_uniform", trainable=True)
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer="zeros", trainable=True)
        super().build(batch_input_shape)

    def call(self, inputs):
        return self.activation(tf.matmul(inputs, self.kernel) + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return (batch_input_shape[0], self.units)

    def get_config(self):
        base_config = super().get_config().copy()
        return {**base_config, "units": self.units, "activation": self.activation}

### 创建一个具有多个输入的Concatenate层

In [20]:
class Concatenate(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        return [X1 + X2, X1 * X2, X1 / X2]

    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1, b1, b1]

### 定义一个训练期间添加高斯噪声测试期间什么都不做的

In [21]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwarge):
        super().__init__()
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), mean=0.0, stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

## 12.3.6 自定义模型

### 创建一个跳过链接或者包含循环的model

In [22]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu", kernel_initializer="he_normal") for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)

        return inputs + Z

In [23]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)


## 12.3.7 基于model内部的损失和指标

In [24]:
class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",kernel_initializer="lecun_normal") for _ in range(5)]

        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)

## 12.3.8 使用自动微分计算梯度

## 12.3.9 使用自定义训练循环
### 编写自定义训练循环会使代码更长 更加容易出错并且更加容易出错

### 创建一个简单的模型

In [25]:
l2_reg = keras.regularizers.l2(0.001)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_initializer = l2_reg)
])

### 创建一个函数用来对数据进行随机取样

In [ ]:
def random_batch(X, y, batch_size):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

### 定义一个函数用来显示训练状态 包括步数 总步数 从轮次开始以来的平均损失

In [ ]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in metrics])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - loss: {:.4f} - {}".format(iteration, total, loss, metrics), end=end)

### 定义一些超参数 然后选择优化器 损失函数 指标

In [27]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
loss_fn = keras.losses.MeanSquaredError()
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

### 定义训练循环

In [28]:

for epoch in range(n_epochs):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(n_steps):
        X_batch, y_batch = random_batch(X_train, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch,training= True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = main_loss + sum(model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step, n_steps, mean_loss.result(), metrics)
    print_status_bar(step, n_steps, mean_loss.result(), metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()


Epoch 0/5


NameError: name 'random_batch' is not defined